In [2]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
import tensorflow as tf
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
sess = tf.InteractiveSession()
x = tf.placeholder('float',shape=[None,784])
y_ = tf.placeholder('float',shape=[None,10])
w = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.initialize_all_variables())
y = tf.nn.softmax(tf.matmul(x,w)+b)
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x:batch[0],y_:batch[1]})

correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,'float'))

print(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels}))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x10aa5f358>>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 140, in __del__
    self.close()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 905, in close
    self._default_session.__exit__(None, None, None)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/contextlib.py", line 88, in __exit__
    next(self.gen)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/ops.py", line 3215, in get_controller
    assert self.stack[-1] is default
AssertionError: 


0.9092


In [7]:
import tensorflow as tf
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
sess = tf.InteractiveSession()
x = tf.placeholder('float',shape=[None,784])
y_ = tf.placeholder('float',shape=[None,10])
w = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

# 构建一个多层卷积网络，需要创建大量的权重和偏置值，
# 初始化中加入少量噪声打破对称性以避免0梯度。
# 使用ReLU神经元，较小的正数初始化偏置量，以避免神经元节点输出恒为0
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

# 卷积使用1步长stride size，0边距padding size的模板，保证输入输出是同一个大小
# 池化用简单传统的2*2大小的模板做max pooling，抽象成一个函数
def conv2d(x,w):
    return tf.nn.conv2d(x,w, strides=[1,1,1,1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
# 第一层卷积，在每个5*5的patch中算出32个特征，卷积的权重张量是[5,5,1,32]
# 5，5是patch大小，1是输入的通道数目，32是输出的通道数目，每个输出通道有对应的偏置量
w_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
# 为了用这一层，我们把x变成一个4d向量，2、3维图片宽高，4维代表图片的颜色通道grey1,rgb3
# 把x_image和权值向量进行卷积，加上偏置项，然后应用ReLU激活函数，最后池化
x_image = tf.reshape(x,[-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
# 第二层卷积，构建更深层的网络，把几个类似的层堆叠起来，第二层中，64个特征
w_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
# 密集连接层，图片尺寸减小到7*7，加入1024个神经元的全连接层，处理图片
# 把池化层输出的张量reshape成一些向量，乘权重加偏置，最后ReLU
w_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)
# Dropout 减少过度拟合，在输出层前加。用占位符代表神经元的输出在dropout中
# 保持不变的概率，可以在训练中启用的dropout，测试中关闭。
keep_prob = tf.placeholder('float')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
# 输出层
w_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)
# train and evaluation
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,'float'))
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0],y_:batch[1],keep_prob:1.0})
        print('step %d, trainning accuracy %g' % (i,train_accuracy))
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})
print('test accuracy %g'% accuracy.eval(feed_dict={
    x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x10b5fde48>>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 140, in __del__
    self.close()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 905, in close
    self._default_session.__exit__(None, None, None)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/contextlib.py", line 88, in __exit__
    next(self.gen)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/ops.py", line 3215, in get_controller
    assert self.stack[-1] is default
AssertionError: 


step 0, trainning accuracy 0.1
step 100, trainning accuracy 0.86
step 200, trainning accuracy 0.9
step 300, trainning accuracy 0.82
step 400, trainning accuracy 0.96
step 500, trainning accuracy 0.9
step 600, trainning accuracy 1
step 700, trainning accuracy 0.96
step 800, trainning accuracy 0.86
step 900, trainning accuracy 1
step 1000, trainning accuracy 0.98
step 1100, trainning accuracy 0.94
step 1200, trainning accuracy 0.94
step 1300, trainning accuracy 0.98
step 1400, trainning accuracy 0.96
step 1500, trainning accuracy 0.96
step 1600, trainning accuracy 0.98
step 1700, trainning accuracy 0.98
step 1800, trainning accuracy 0.94
step 1900, trainning accuracy 1
step 2000, trainning accuracy 1
step 2100, trainning accuracy 0.86
step 2200, trainning accuracy 0.98
step 2300, trainning accuracy 0.98
step 2400, trainning accuracy 0.96
step 2500, trainning accuracy 1
step 2600, trainning accuracy 0.96
step 2700, trainning accuracy 0.96
step 2800, trainning accuracy 1
step 2900, trainni

In [64]:
print('%.2f'%train_accuracy)
print(y_conv.consumers())
print(y_conv.graph)
print(y_conv.name)
batch[0]

1.00
[<tensorflow.python.framework.ops.Operation object at 0x11c385d68>, <tensorflow.python.framework.ops.Operation object at 0x11c3c1d68>, <tensorflow.python.framework.ops.Operation object at 0x113c2f4e0>, <tensorflow.python.framework.ops.Operation object at 0x113c3e390>, <tensorflow.python.framework.ops.Operation object at 0x11c55aa90>]
Softmax_2:0


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [72]:
h_fc1

<tf.Tensor 'Relu_9:0' shape=(?, 1024) dtype=float32>